# Static ZZ estimation
In this notebook we estimate the always-on static ZZ strength between two transmons coupled through a resonant cavity. We compare numerical diagonalization and a perturbative analytical result. The two transmons are coupled to the resonator a Jaynes-Cummmings interaction.

In [81]:
import numpy as np
import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm
from qutip import tensor, destroy, qeye, num
# Define system parameters (in GHz)
EC1 = 0.250  # Charging energy of qubit 1 (GHz)
EC2 = 0.220  # Charging energy of qubit 2 (GHz)
EJ1 = 15.0   # Josephson energy of qubit 1 (GHz)
EJ2 = 13.9   # Josephson energy of qubit 2 (GHz)
freq_resonator = 7.0  # Resonator frequency (GHz)
g1 = 0.04  # Coupling strength between qubit 1 and resonator (GHz)
g2 = 0.03  # Coupling strength between qubit 2 and resonator (GHz)
ncutoff = 3  # Charge basis cutoff for transmons
truncation_level = 3  # Truncate the Hilbert space for easier diagonalization

# Define Transmon qubits and the resonator
qubit1 = scq.Transmon(EJ=EJ1, EC=EC1, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
qubit2 = scq.Transmon(EJ=EJ2, EC=EC2, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
resonator = scq.Oscillator(E_osc=freq_resonator, truncated_dim=truncation_level)

# Define the Hilbert space
hilbertspace = HilbertSpace([qubit1, qubit2, resonator])
N = 3  # Number of resonator photon states (Fock basis)
levels_q1 = 3  # Number of transmon levels for qubit 1
levels_q2 = 3  # Number of transmon levels for qubit 2

# Operators for the resonator
a = tensor(qeye(levels_q1), qeye(levels_q2), destroy(N))  # resonator annihilation operator

# Operators for transmon qubit 1 (as a multi-level system)
n_q1 = tensor(num(levels_q1), qeye(levels_q2), qeye(N))  # Number operator for qubit 1
sm_q1 = tensor(destroy(levels_q1), qeye(levels_q2), qeye(N))  # Lowering operator for qubit 1

# Operators for transmon qubit 2 (as a multi-level system)
n_q2 = tensor(qeye(levels_q1), num(levels_q2), qeye(N))  # Number operator for qubit 2
sm_q2 = tensor(qeye(levels_q1), destroy(levels_q2), qeye(N))  # Lowering operator for qubit 2

# Interaction Hamiltonians (Jaynes-Cummings type)
H_int1 = g1 * (sm_q1 * a.dag() + sm_q1.dag() * a)  # Interaction between qubit 1 and resonator
H_int2 = g2 * (sm_q2 * a.dag() + sm_q2.dag() * a)  # Interaction between qubit 2 and resonator

hilbertspace.add_interaction(
    qobj = H_int1 + H_int2
)
hilbertspace.generate_lookup()

# Calculate energy differences for qubit states
E_00 = hilbertspace.energy_by_bare_index((0,0,0), subtract_ground = True)  # Ground state (|00⟩)
E_01 = hilbertspace.energy_by_bare_index((0,1,0), subtract_ground = True)  # Qubit 1 in ground state, qubit 2 in excited state (|01⟩)
E_10 = hilbertspace.energy_by_bare_index((1,0,0), subtract_ground = True)  # Qubit 1 in excited state, qubit 2 in ground state (|10⟩)
E_11 = hilbertspace.energy_by_bare_index((1,1,0), subtract_ground = True)  # Both qubits in excited state (|11⟩)


# Calculate the ZZ interaction
ZZ_interaction = (E_11 - E_10 - E_01)

# Display the results
print(f"Static ZZ interaction: {ZZ_interaction} ")  # Convert from GHz to MHz


Static ZZ interaction: 3.2936872358391156e-06 


In [82]:
def ZZ(J, d1, d2, D, W):
    factor = J**2/2/(d1 + D)**2
    A = 2*(d1 + D)*(d1 + d2)/(D - d2)
    B =  ( (d1**3 - 2*d1*D**2 - 2*D**3)/(d1*D**2*(d2-D))
          + 1/2*(4*(3*d1+D)*(d1**2+d1*D+D**2)/(D**2*(2*d1+D)**2) - 16*D/(3*d1**2+8*d1*D+4*D**2) )
          + 2*d1/(D*d2)
          - 2*(d1+D)/(d1+D-d2)**2
          - 2*(d1+D)/d1/(d1+D-d2)

         )*W**2
    
    return (factor*A, factor*B )

def ZZ_alt(J, d1, d2, D, W):
    xi = - J**2 * (d1 + d2) / (D+d1) / (d2 - D)
    return xi 

In [83]:
w1 = hilbertspace.energy_by_bare_index((1,0,0), subtract_ground = True)
w2 = hilbertspace.energy_by_bare_index((0,1,0), subtract_ground = True)
D = w1 - w2
d1 = qubit1.anharmonicity()
d2 = qubit2.anharmonicity()
J = 0.5 * g1*g2*(w1 + w2 - 2*freq_resonator)/ (w1 - freq_resonator) / (w2 - freq_resonator)
ZZ(J, d1, d2, D, 0)[0]

1.40323998651981e-06

In [84]:
ZZ_alt(J, d1, d2, D, 0)

1.4032399865198099e-06

In [85]:
import numpy as np

def calculate_ZZ_interaction(J, delta1, delta2, omega1, omega2):
    """
    Estimate the ZZ interaction strength between two qubits.
    
    Parameters:
    J       : Exchange coupling between qubits (in Hz)
    delta1  : Anharmonicity of the first transmon (in Hz)
    delta2  : Anharmonicity of the second transmon (in Hz)
    omega1  : Frequency of the first qubit (in Hz)
    omega2  : Frequency of the second qubit (in Hz)
    
    Returns:
    xi_ZZ   : Estimated ZZ interaction strength (in Hz)
    """
    # Calculate qubit detuning (difference in frequencies)
    delta_omega = omega1 - omega2
    
    # Calculate the ZZ interaction strength using the perturbative formula
    xi_ZZ = - (J**2 * (delta1 + delta2)) / ((delta_omega + delta1) * (delta2 - delta_omega))
    
    return xi_ZZ


In [86]:
def calculate_J(g1, g2, omega1, omega2, omega_r):
    """
    Calculate the exchange coupling J between two transmons.
    
    Parameters:
    g1       : Coupling strength between the first transmon and the resonator (in Hz)
    g2       : Coupling strength between the second transmon and the resonator (in Hz)
    omega1   : Frequency of the first transmon (in Hz)
    omega2   : Frequency of the second transmon (in Hz)
    omega_r  : Frequency of the resonator (in Hz)
    
    Returns:
    J        : Exchange coupling (in Hz)
    """
    numerator = g1 * g2 * (omega1 + omega2 - 2 * omega_r)
    denominator = 2 * (omega1 - omega_r) * (omega2 - omega_r)
    
    # Calculate the exchange coupling J
    J = numerator / denominator
    return J

# Example values (in Hz)
g1 = 0.05      # Coupling strength between the first transmon and resonator (Hz)
g2 = 0.06     # Coupling strength between the second transmon and resonator (Hz)
omega1 = w1  # Frequency of the first transmon (Hz)
omega2 = w2   # Frequency of the second transmon (Hz)
omega_r = freq_resonator   # Frequency of the resonator (Hz)

# Calculate the exchange coupling J
J = calculate_J(g1, g2, omega1, omega2, omega_r)


print(f"Exchange coupling J: {J:.2e} Hz")
zz_interaction = calculate_ZZ_interaction(J, d1, d2, omega1, omega2)
print(f"Estimated ZZ interaction strength: {zz_interaction:.2e} Hz")

Exchange coupling J: -1.75e-03 Hz
Estimated ZZ interaction strength: 8.77e-06 Hz


In [87]:
import numpy as np
from scipy.linalg import eigh

# Define system parameters
omega_t1 = w1  # Frequency of first transmon in GHz
omega_t2 = w2  # Frequency of second transmon in GHz
omega_r = freq_resonator   # Resonator frequency in GHz
anharmonicity1 = d1  # Anharmonicity of both transmons
anharmonicity2 = d2
# Define the number of energy levels to consider for each system
n_levels = 3

# Create operators for the transmon and resonator
def create_operators(n):
    a = np.diag(np.sqrt(np.arange(1, n)), 1)  # Lowering operator
    n_op = np.diag(np.arange(n))  # Number operator
    return a, n_op

a_t1, n_t1 = create_operators(n_levels)  # Transmon 1 operators
a_t2, n_t2 = create_operators(n_levels)  # Transmon 2 operators
a_r, n_r = create_operators(n_levels)    # Resonator operators

# Create identity matrices
I_t1 = np.eye(n_levels)
I_t2 = np.eye(n_levels)
I_r = np.eye(n_levels)

# Define Hamiltonians for each subsystem
H_t1 = omega_t1 * np.kron(np.kron(n_t1, I_t2), I_r) + (anharmonicity1/ 2) * np.kron(np.kron(n_t1 @ (n_t1 - I_t1), I_t2), I_r)
H_t2 = omega_t2 * np.kron(np.kron(I_t1, n_t2), I_r) + (anharmonicity2 / 2) * np.kron(np.kron(I_t1, n_t2 @ (n_t2 - I_t2)), I_r)
H_r = omega_r * np.kron(np.kron(I_t1, I_t2), n_r)

# Interaction terms (Jaynes-Cummings type coupling)
H_int1 = g1 * (np.kron(np.kron(a_t1 + a_t1.T, I_t2), a_r + a_r.T))
H_int2 = g2 * (np.kron(np.kron(I_t1, a_t2 + a_t2.T), a_r + a_r.T))

# Total Hamiltonian
H_total = H_t1 + H_t2 + H_r + H_int1 + H_int2

# Diagonalize the full Hamiltonian
eigenvalues, eigenstates = eigh(H_total)

# Print the lowest few eigenvalues
print("Lowest eigenvalues (energy levels):", eigenvalues[:5])


Lowest eigenvalues (energy levels): [-5.00227611e-04  4.99561750e+00  5.49271004e+00  7.00245306e+00
  1.02532156e+01]


In [88]:
eigenvalues - eigenvalues[0]

array([ 0.        ,  4.99611773,  5.49321027,  7.00295329, 10.25371583,
       10.48935416, 11.22145054, 11.99962496, 12.49678645, 14.00739874,
       15.74690811, 16.21760404, 17.25167536, 17.4935087 , 18.21946123,
       19.01079455, 19.50736795, 21.47516201, 22.74555405, 23.21623606,
       24.26241419, 24.51080428, 25.23007544, 28.46826993, 29.76230757,
       30.23361967, 35.48520809])

In [89]:
hilbertspace.eigenvals() - hilbertspace.eigenvals()[0]

array([ 0.        ,  4.99820832,  5.49505288,  7.0015122 , 10.25744445,
       10.4932645 ])

In [90]:
import numpy as np
from scipy.linalg import eigh

# Define system parameters
omega_t1 = 5.0  # Frequency of first transmon in GHz
omega_t2 = 4.9  # Frequency of second transmon in GHz
omega_r = 14.3   # Resonator frequency in GHz
g1 = 0.01        # Coupling strength of transmon 1 to resonator
g2 = 0.011       # Coupling strength of transmon 2 to resonator
anharmonicity = -0.33  # Anharmonicity of both transmons

# Define the number of energy levels to consider for each system
n_levels = 5

# Create operators for the transmon and resonator
def create_operators(n):
    a = np.diag(np.sqrt(np.arange(1, n)), 1)  # Lowering operator
    n_op = np.diag(np.arange(n))  # Number operator
    return a, n_op

a_t1, n_t1 = create_operators(n_levels)  # Transmon 1 operators
a_t2, n_t2 = create_operators(n_levels)  # Transmon 2 operators
a_r, n_r = create_operators(n_levels)    # Resonator operators

# Create identity matrices
I_t1 = np.eye(n_levels)
I_t2 = np.eye(n_levels)
I_r = np.eye(n_levels)

# Define Hamiltonians for each subsystem
H_t1 = omega_t1 * np.kron(np.kron(n_t1, I_t2), I_r) + (anharmonicity / 2) * np.kron(np.kron(n_t1 @ (n_t1 - I_t1), I_t2), I_r)
H_t2 = omega_t2 * np.kron(np.kron(I_t1, n_t2), I_r) + (anharmonicity / 2) * np.kron(np.kron(I_t1, n_t2 @ (n_t2 - I_t2)), I_r)
H_r = omega_r * np.kron(np.kron(I_t1, I_t2), n_r)

# Interaction terms (Jaynes-Cummings type coupling)
H_int1 = g1 * (np.kron(np.kron(a_t1 + a_t1.T, I_t2), a_r + a_r.T))
H_int2 = g2 * (np.kron(np.kron(I_t1, a_t2 + a_t2.T), a_r + a_r.T))

# Total Hamiltonian
H_total = H_t1 + H_t2 + H_r + H_int1 + H_int2

# Diagonalize the full Hamiltonian
eigenvalues, eigenstates = eigh(H_total)

# Extract relevant energy eigenvalues corresponding to |00>, |01>, |10>, |11> states
# Assuming that the lowest energy states correspond to the computational basis |00>, |01>, |10>, |11>
E00 = eigenvalues[0]  # Ground state (|00>)
E01 = eigenvalues[1]  # First excited state (|01>)
E10 = eigenvalues[2]  # Second excited state (|10>)
E11 = eigenvalues[3]  # Third excited state (|11>)

# Calculate the ZZ interaction from energy differences
ZZ_exact = (E11 - E10) - (E01 - E00)

print("ZZ coupling (Exact Diagonalization):", ZZ_exact)

# Analytical ZZ coupling based on the paper's formula (Eq. 4.5)
delta1 = anharmonicity
delta2 = anharmonicity
Delta1 = omega_t1 - omega_r
Delta2 = omega_t2 - omega_r
J = g1 * g2 * (omega_t1 + omega_t2 - 2 * omega_r) / (2 * (omega_t1 - omega_r) * (omega_t2 - omega_r))

ZZ_analytical = -J**2 * (delta1 + delta2) / ((Delta1 + delta1) * (delta2 - Delta1))

print("ZZ coupling (Analytical):", ZZ_analytical)


ZZ coupling (Exact Diagonalization): -0.43000264100433583
ZZ coupling (Analytical): -1.0575774769112515e-12


In [91]:
import numpy as np
from qutip import *

# System parameters
omega_r = 7.0  # Resonator frequency (GHz)
omega_q1 = 4.0  # Qubit 1 frequency (GHz)
omega_q2 = 4.1  # Qubit 2 frequency (GHz)
g1 = 0.02  # Coupling strength between resonator and qubit 1 (GHz)
g2 = 0.012  # Coupling strength between resonator and qubit 2 (GHz)
alpha1 = 0.3  # Anharmonicity for qubit 1 (GHz)
alpha2 = 0.35  # Anharmonicity for qubit 2 (GHz)

N = 3  # Number of resonator photon states (Fock basis)
levels_q1 = 5  # Number of transmon levels for qubit 1
levels_q2 = 5  # Number of transmon levels for qubit 2

# Operators for the resonator
a = tensor(destroy(N), qeye(levels_q1), qeye(levels_q2))  # resonator annihilation operator

# Operators for transmon qubit 1 (as a multi-level system)
n_q1 = tensor(qeye(N), num(levels_q1), qeye(levels_q2))  # Number operator for qubit 1
sm_q1 = tensor(qeye(N), destroy(levels_q1), qeye(levels_q2))  # Lowering operator for qubit 1

# Operators for transmon qubit 2 (as a multi-level system)
n_q2 = tensor(qeye(N), qeye(levels_q1), num(levels_q2))  # Number operator for qubit 2
sm_q2 = tensor(qeye(N), qeye(levels_q1), destroy(levels_q2))  # Lowering operator for qubit 2

# Hamiltonian terms

# Resonator energy: H_r = hbar * omega_r * a† * a
H_r = omega_r * a.dag() * a

# Transmon qubit 1 energy including anharmonicity: H_q1 = omega_q1 * n_q1 - alpha1 * n_q1 * (n_q1 - 1)
H_q1 = omega_q1 * n_q1 - 0.5 * alpha1 * n_q1 * (n_q1 - 1)

# Transmon qubit 2 energy including anharmonicity: H_q2 = omega_q2 * n_q2 - alpha2 * n_q2 * (n_q2 - 1)
H_q2 = omega_q2 * n_q2 - 0.5 * alpha2 * n_q2 * (n_q2 - 1)

# Interaction Hamiltonians (Jaynes-Cummings type)
H_int1 = g1 * (sm_q1 * a.dag() + sm_q1.dag() * a)  # Interaction between qubit 1 and resonator
H_int2 = g2 * (sm_q2 * a.dag() + sm_q2.dag() * a)  # Interaction between qubit 2 and resonator

# Total Hamiltonian
H_total = H_r + H_q1 + H_q2 + H_int1 + H_int2

eig = H_total.eigenenergies()
eig = eig - eig[0]



In [92]:
eig

array([ 0.        ,  3.99986661,  4.09995041,  7.00018298,  7.69975756,
        7.84991134,  8.0998171 , 11.00002535, 11.09966666, 11.1001227 ,
       11.24987998, 11.79970753, 11.84977852, 14.00036595, 14.19958974,
       14.29985416, 14.69989815, 14.85007606, 15.09996517, 15.19961696,
       15.2497466 , 15.54966916, 18.00058382, 18.0997933 , 18.10044412,
       18.25003914, 18.29954004, 18.29972079, 18.79983747, 18.84991904,
       18.94940569, 18.94981008, 21.19922931, 21.2998417 , 21.70076573,
       21.85050642, 21.99961162, 22.04950097, 22.10066228, 22.19973297,
       22.24988158, 22.34954699, 22.54979156, 25.10091951, 25.25055813,
       25.29916899, 25.29968416, 25.39951965, 25.44947094, 25.8008421 ,
       25.85072654, 25.94951781, 25.9499233 , 28.2000992 , 28.30026655,
       28.4994439 , 28.99955688, 29.04912239, 29.20099721, 29.25077603,
       29.34964994, 29.55090719, 32.30017696, 32.30048454, 32.39945099,
       32.44908685, 32.95060627, 32.95141245, 35.49888816, 36.00

In [93]:
eig[6] - eig[2] - eig[1]

7.6470259902095e-08

In [94]:
# Calculate the exchange coupling J
J = calculate_J(g1, g2, omega_q1, omega_q2, omega_r)
zz_interaction = calculate_ZZ_interaction(J, -alpha1, -alpha2, omega_q1, omega_q2)
zz_interaction

4.304684898929843e-08

In [126]:
import numpy as np
import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm
from qutip import *
# Define system parameters (in GHz)
EC1 = 0.250  # Charging energy of qubit 1 (GHz)
EC2 = 0.220  # Charging energy of qubit 2 (GHz)
EJ1 = 15.0   # Josephson energy of qubit 1 (GHz)
EJ2 = 13.9   # Josephson energy of qubit 2 (GHz)
freq_resonator = 8.5  # Resonator frequency (GHz)
g1 = 0.07  # Coupling strength between qubit 1 and resonator (GHz)
g2 = 0.08  # Coupling strength between qubit 2 and resonator (GHz)
ncutoff = 7  # Charge basis cutoff for transmons
truncation_level =7  # Truncate the Hilbert space for easier diagonalization

# Define Transmon qubits and the resonator
qubit1 = scq.Transmon(EJ=EJ1, EC=EC1, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
qubit2 = scq.Transmon(EJ=EJ2, EC=EC2, ng=0.0, ncut=ncutoff, truncated_dim=truncation_level)
resonator = scq.Oscillator(E_osc=freq_resonator, truncated_dim=truncation_level)

# Define the Hilbert space
hilbertspace = HilbertSpace([qubit1, qubit2, resonator])
N = 7  # Number of resonator photon states (Fock basis)
levels_q1 = 7  # Number of transmon levels for qubit 1
levels_q2 = 7  # Number of transmon levels for qubit 2
# Operators for the resonator
a = tensor(qeye(levels_q1), qeye(levels_q2), destroy(N))  # resonator annihilation operator

# Operators for transmon qubit 1 (as a multi-level system)
n_q1 = tensor(num(levels_q1), qeye(levels_q2), qeye(N))  # Number operator for qubit 1
sm_q1 = tensor(destroy(levels_q1), qeye(levels_q2), qeye(N))  # Lowering operator for qubit 1

# Operators for transmon qubit 2 (as a multi-level system)
n_q2 = tensor(qeye(levels_q1), num(levels_q2), qeye(N))  # Number operator for qubit 2
sm_q2 = tensor(qeye(levels_q1), destroy(levels_q2), qeye(N))  # Lowering operator for qubit 2

# Interaction Hamiltonians (Jaynes-Cummings type)
H_int1 = g1 * (sm_q1 * a.dag() + sm_q1.dag() * a)
H_int2 = g2 * (sm_q2 * a.dag() + sm_q2.dag() * a)  # Interaction between qubit 2 and resonator

# Add interactions to the Hilbert space
hilbertspace.add_interaction(qobj=H_int1 + H_int2)
hilbertspace.generate_lookup()

# Calculate energy differences for qubit states
E_00 = hilbertspace.energy_by_bare_index((0, 0, 0), subtract_ground=True)  # Ground state (|00⟩)
E_01 = hilbertspace.energy_by_bare_index((0, 1, 0), subtract_ground=True)  # |01⟩ state
E_10 = hilbertspace.energy_by_bare_index((1, 0, 0), subtract_ground=True)  # |10⟩ state
E_11 = hilbertspace.energy_by_bare_index((1, 1, 0), subtract_ground=True)  # |11⟩ state

E_02 = hilbertspace.energy_by_bare_index((0, 2, 0), subtract_ground=True)
E_20 = hilbertspace.energy_by_bare_index((2, 0, 0), subtract_ground=True)

# Calculate the ZZ interaction numerically
ZZ_interaction_numerical = (E_11 - E_10 - E_01)

# Analytical ZZ formula
delta1 = - E_10 + (E_20 - E_10)  # Anharmonicity of qubit 1 (GHz)
delta2 = - E_01 + (E_02 - E_01)  # Anharmonicity of qubit 2 (GHz)
w1 = E_10
w2 = E_01
# J = calculate_J(g1, g2, w1, w2, freq_resonator)
# zz_interaction = calculate_ZZ_interaction(J, delta1, delta2, w1, w2)

# Compare the results
print(f"Numerical ZZ interaction: {ZZ_interaction_numerical} GHz")
# print(f"Analytical ZZ interaction: {zz_interaction*2} GHz")


Numerical ZZ interaction: -1.5264212915866437e-05 GHz


In [127]:
a1 = qubit1.anharmonicity() #E_20 - 2*E_10 
a2 = qubit2.anharmonicity() 
w1 = qubit1.E01()
w2 = qubit2.E01()
D = w1 - w2
J = calculate_J(g1, g2, w1, w2, freq_resonator)
zz = ZZ_1(J, a1, a2, D, 0)
print(zz)

-1.665094500980164e-05


In [99]:
ZZ_1(J, a1, a2, D, 0)

-7.969901358296874e-08

In [110]:
ZZ_2(J, a1, a2, D, 0)

(9.264378381400333e-08, 0.0)

In [109]:
def ZZ_1(J, d1, d2, D, W):
    zz = 2*J**2 * (d1 + d2) / (D + d1) / (D - d2)
    
    return zz

def calculate_J(g1, g2, omega1, omega2, omega_r):
    numerator = g1 * g2 * (omega1 + omega2 - 2 * omega_r)
    denominator = 2 * (omega1 - omega_r) * (omega2 - omega_r)
    J = numerator / denominator
    return J

In [98]:
def ZZ_2(J, d1, d2, D, W):
    factor = J**2/2/(d1 + D)**2
    A = 2*(d1 + D)*(d1 + d2)/(D - d2)
    B =  ( (d1**3 - 2*d1*D**2 - 2*D**3)/(d1*D**2*(d2-D))
          + 1/2*(4*(3*d1+D)*(d1**2+d1*D+D**2)/(D**2*(2*d1+D)**2) - 16*D/(3*d1**2+8*d1*D+4*D**2) )
          + 2*d1/(D*d2)
          - 2*(d1+D)/(d1+D-d2)**2
          - 2*(d1+D)/d1/(d1+D-d2)

         )*W**2
    
    return (factor*A, factor*B )
